In [14]:
import math
import cv2
import cvzone
import torch


import easyocr
import csv
import os
import numpy as np
import pandas as pd
import cv2
import re

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath


In [17]:
os.chdir("..")

In [15]:
def predict_number_plate(img, ocr):
    result = ocr.ocr(img, cls=True)
    result = result[0]
    texts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    if (scores[0]*100) >= 90:
        return re.sub(r'[^a-zA-Z0-9]', '', texts[0]), scores[0]
    else:
        return None, None

In [58]:
cap = cv2.VideoCapture("train/images/new53.jpg")  # For videos

model =  torch.hub.load('./yolov5', 'custom', source ='local', path='best.pt',force_reload=False)

device = torch.device("mps") # change to cuda for windows gpu

classNames = ["with helmet", "without helmet", "rider", "number plate"]
num = 0
old_npconf = 0

# grab the width, height, and fps of the frames in the video stream.
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# initialize the FourCC and a video writer object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))


ocr = easyocr.Reader(['en']) # need to run only once to download and load model into memory

while True:
    success, img = cap.read()
    # Check if the frame was read successfully
    if not success:
        break
    new_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = []
    results = model([new_img])
    result.append(results)
    for r in result:
        print(r)


YOLOv5  2024-4-26 Python-3.10.0 torch-2.3.0+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


image 1/1: 1800x1036 3 without helmets, 2 riders, 2 number plates
Speed: 7.8ms pre-process, 179.9ms inference, 0.0ms NMS per image at shape (1, 3, 640, 384)
image 1/1: 708x406 1 with helmet, 1 number plate
Speed: 4.0ms pre-process, 168.0ms inference, 0.0ms NMS per image at shape (1, 3, 640, 384)
image 1/1: 1064x954 1 with helmet, 1 number plate
Speed: 4.5ms pre-process, 251.7ms inference, 1.3ms NMS per image at shape (1, 3, 640, 576)


In [59]:
results.names

{0: 'with helmet', 1: 'without helmet', 2: 'rider', 3: 'number plate'}

In [60]:
results.xyxy

[tensor([[3.94783e+02, 7.77288e+02, 5.49848e+02, 8.64148e+02, 7.56493e-01, 3.00000e+00],
         [5.89161e+02, 1.04086e+02, 7.68846e+02, 2.20175e+02, 4.82333e-01, 0.00000e+00]])]

In [61]:
results.xywh

[tensor([[4.72316e+02, 8.20718e+02, 1.55065e+02, 8.68602e+01, 7.56493e-01, 3.00000e+00],
         [6.79004e+02, 1.62130e+02, 1.79686e+02, 1.16090e+02, 4.82333e-01, 0.00000e+00]])]

In [62]:
results.xywhn

[tensor([[0.49509, 0.77135, 0.16254, 0.08164, 0.75649, 3.00000],
         [0.71174, 0.15238, 0.18835, 0.10911, 0.48233, 0.00000]])]

In [63]:
results.xyxyn

[tensor([[0.41382, 0.73053, 0.57636, 0.81217, 0.75649, 3.00000],
         [0.61757, 0.09782, 0.80592, 0.20693, 0.48233, 0.00000]])]